<a href="https://colab.research.google.com/github/fabianhuertas1992/Prefactibilidad_SUAN/blob/main/Script/Recorte_Poligonos_GeoJSON_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acceso a Drive de Google - Librerias

In [ ]:
# @title Montar acceso a Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Instala librerias
!pip install rasterio
!pip install geopandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 48.8 MB/s eta 0:00:00


# Recorte de imágenes

In [ ]:
# @title Recorte de Imágenes tomadas de GeoJson
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import re

def procesar_imagenes_y_poligonos(geojson_dir, imagenes_dir, salida_dir, imagenes_bandas):
    os.makedirs(salida_dir, exist_ok=True)

    # Obtener lista de archivos GeoJSON en la carpeta
    geojson_files = [f for f in os.listdir(geojson_dir) if f.endswith(".geojson")]

    for geojson_file in geojson_files:
        geojson_path = os.path.join(geojson_dir, geojson_file)
        poligonos = gpd.read_file(geojson_path)

        # Extraer información del geojson
        match = re.match(r"([^_]+)_([^_]+)_(\d+\.\d+)", geojson_file)
        if not match:
            print(f"Advertencia: No se pudo extraer información del nombre {geojson_file}, omitiendo.")
            continue

        departamento, municipio, shape_id = match.groups()
        municipio = municipio.replace(" ", "_")  # Reemplazar espacios en el nombre del municipio

        for imagen, bandas in imagenes_bandas.items():
            for archivo_banda in bandas:
                banda_path = os.path.join(imagenes_dir, archivo_banda)

                if not os.path.exists(banda_path):
                    print(f"Advertencia: No se encontró {banda_path}, omitiendo.")
                    continue

                try:
                    with rasterio.open(banda_path) as src:
                        profile = src.profile  # Guardar el perfil del raster original

                        for idx, row in poligonos.iterrows():
                            geom = [row["geometry"]]  # Obtener la geometría del polígono

                            # Extraer información de la imagen de entrada
                            fecha, satelite, banda = os.path.splitext(archivo_banda)[0].split("_")[-3:]

                            # Crear nombre de salida con el formato correcto
                            nombre_salida = f"{departamento}_{shape_id}_{fecha}_{municipio}_{satelite}_{banda}.tiff"
                            nombre_salida_path = os.path.join(salida_dir, nombre_salida)

                            try:
                                # Recortar el raster usando el polígono
                                out_image, out_transform = mask(src, geom, crop=True)

                                # Crear el perfil de salida
                                out_meta = src.meta.copy()
                                out_meta.update({
                                    "driver": "GTiff",
                                    "height": out_image.shape[1],
                                    "width": out_image.shape[2],
                                    "transform": out_transform
                                })

                                # Guardar el archivo recortado
                                with rasterio.open(nombre_salida_path, "w", **out_meta) as dst:
                                    dst.write(out_image)

                                print(f"Recorte guardado: {nombre_salida_path}")
                            except Exception as e:
                                print(f"Error al recortar {banda_path} con {geojson_file}: {e}")
                except Exception as e:
                    print(f"Error al abrir la imagen {banda_path}: {e}")

# Definir directorios
geojson_directory = "/content/drive/MyDrive/Localización_por_GeoJson/Municipios_individuales_geojson/francisco"
imagenes_directory = "/content/drive/MyDrive/poligonos_imagenes/imagenes_entrada_sentinel/francisco/posterior"
salida_directory = "/content/drive/MyDrive/poligonos_imagenes/imagenes_salida_sentinel/francisco/posterior"

# Diccionario de imágenes con sus respectivas bandas
imagenes_bandas = {
     "imagen 1": [
        "2025-02-24_sentinel-2_B03.tiff",
        "2025-02-24_sentinel-2_B04.tiff",
        "2025-02-24_sentinel-2_B08.tiff",
        "2025-02-24_sentinel-2_B011.tiff"
    ],
}

# Ejecutar la función para procesar imágenes y polígonos
procesar_imagenes_y_poligonos(geojson_directory, imagenes_directory, salida_directory, imagenes_bandas)


Recorte guardado: /content/drive/MyDrive/poligonos_imagenes/imagenes_salida_sentinel/francisco/posterior/Narino_650.655162467_2025-02-24_Francisco_Pizarro_sentinel-2_B03.tiff
Recorte guardado: /content/drive/MyDrive/poligonos_imagenes/imagenes_salida_sentinel/francisco/posterior/Narino_650.655162467_2025-02-24_Francisco_Pizarro_sentinel-2_B04.tiff
Recorte guardado: /content/drive/MyDrive/poligonos_imagenes/imagenes_salida_sentinel/francisco/posterior/Narino_650.655162467_2025-02-24_Francisco_Pizarro_sentinel-2_B08.tiff
Recorte guardado: /content/drive/MyDrive/poligonos_imagenes/imagenes_salida_sentinel/francisco/posterior/Narino_650.655162467_2025-02-24_Francisco_Pizarro_sentinel-2_B011.tiff
Recorte guardado: /content/drive/MyDrive/poligonos_imagenes/imagenes_salida_sentinel/francisco/posterior/Narino_616.273934186_2025-02-24_Francisco_Pizarro_sentinel-2_B03.tiff
Recorte guardado: /content/drive/MyDrive/poligonos_imagenes/imagenes_salida_sentinel/francisco/posterior/Narino_616.27393418